In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
import numpy as np
from scipy.stats import poisson, norm

In [ ]:
def compute_scaling_ratio(mu_drain,mu_demand,drift_sd,init_state):
    drain_time = init_state/(mu_drain-mu_demand)
    accum_std = drift_sd*np.sqrt(drain_time)
    ratio = accum_std/init_state
    
    return ratio

In [ ]:
def compute_workloads(arrival_buffer,inter_buffer,drain_buffer):
    workload_1= arrival_buffer/(mu_drain/2)+(inter_buffer+drain_buffer)/(mu_drain)
    workload_2 = (inter_buffer+arrival_buffer)/(mu_fast)
    return workload_1, workload_2
    
def compute_draining_times(arrival_buffer,inter_buffer,drain_buffer):
    workload_1, workload_2 = compute_workloads(arrival_buffer,inter_buffer,drain_buffer)
    drain_time_1= workload_1/(1-mu_demand*2/mu_drain)
    drain_time_2 = workload_2/(1-mu_demand/mu_fast)
    return drain_time_1, drain_time_2

In [ ]:
def simulate_simple_reentrant_line_ray(arrival_sequence,
                                    transfer_sequence,
                                    fast_sequence,
                                    drain_sequence,
                                    h_thres,ss_thres,
                                    init_state,flow,
                                  init_with_zeros = False):
    arrival_buffer = np.zeros(len(arrival_sequence)+1)
    intermediate_buffer = np.zeros(len(arrival_sequence)+1)
    drain_buffer = np.zeros(len(arrival_sequence)+1)
    arrival_buffer[0] = init_state if not init_with_zeros else 0
    intermediate_buffer[0] = ss_thres#init_state if not init_with_zeros else 0
    drain_buffer[0] = 0#h_thres if not init_with_zeros else 0
    cumulative_arrivals = 0
    cumulative_transfer = 0
    cumulative_fast = 0
    cumulative_drain = 0
    dif_cost = 0
    min_drain_time = np.inf
    
    report = True
    
    completed = False
    for i,(a,t,f,d) in enumerate(zip(arrival_sequence,transfer_sequence,fast_sequence,drain_sequence)):
        if flow:
            t = min(t,arrival_buffer[i]+a)
            f = min(f,intermediate_buffer[i]+t)
            d = min(d,drain_buffer[i]+f)
        else:
            t = min(t,arrival_buffer[i])
            f = min(f,intermediate_buffer[i])
            d = min(d,drain_buffer[i])
            
            
        drain_time_1,drain_time_2 = compute_draining_times(arrival_buffer[i],
                                                           intermediate_buffer[i],
                                                           drain_buffer[i])
        
        min_drain_time = min(min_drain_time,max(drain_time_1+i,drain_time_2+i))
        min_drain_time = max(drain_time_1,drain_time_2)
        
        #print(drain_time_1,drain_time_2)
            
        if (drain_buffer[i] == 0 or intermediate_buffer[i] <= ss_thres) and arrival_buffer[i] > 0:
            d = 0
        else:
            t = 0
            
        #if i % 1000 == 0:
         #   intermediate_buffer[i] = h+100
            
        if intermediate_buffer[i] > h:# and drain_time_1 < min_drain_time:
            t = 0
            #print(arrival_buffer,intermediate_buffer,drain_buffer)
            
           
            
        assert d+t <= 1
        
        #if drain_buffer[i]-intermediate_buffer[i] > 10:#h_thres:
            #f = 0
            
        if drain_buffer[i] > max(1,100000):# and drain_time_2 < min_drain_time:
            #print(drain_buffer[i])
            f = 0    

        if intermediate_buffer[i] == h+10:
            dif_cost += (a-t)*c1+(t-f)*c2+(f-d)*c3
            #print(dif_cost)
                    
        arrival_buffer[i+1] = max(0,arrival_buffer[i]+a-t)
        intermediate_buffer[i+1] = max(0,intermediate_buffer[i]+t-f)
        drain_buffer[i+1] = max(0,drain_buffer[i]+f-d)
        
        cumulative_arrivals += a
        cumulative_transfer += t
        cumulative_fast += f
        cumulative_drain += d
    if report:
        #if False:#i == int(len(arrival_sequence)/4):
        print(cumulative_transfer/cumulative_arrivals)
        print(cumulative_fast/cumulative_arrivals)
        print(cumulative_drain/cumulative_arrivals)
        print()
        
    #print(n_idles,idling_benefit,idling_benefit_2)
    print(dif_cost)
    return arrival_buffer,intermediate_buffer,drain_buffer

In [ ]:
def simulate_simple_reentrant_line(arrival_sequence,
                                    transfer_sequence,
                                    fast_sequence,
                                    drain_sequence,
                                    h_thres,ss_thres,
                                    init_state,flow,
                                  init_with_zeros = False):
    arrival_buffer = np.zeros(len(arrival_sequence)+1)
    intermediate_buffer = np.zeros(len(arrival_sequence)+1)
    drain_buffer = np.zeros(len(arrival_sequence)+1)
    arrival_buffer[0] = 0#init_state if not init_with_zeros else 0
    intermediate_buffer[0] = ss_thres if not init_with_zeros else 0
    drain_buffer[0] = init_state#0#h_thres if not init_with_zeros else 0
    cumulative_arrivals = 0
    cumulative_transfer = 0
    cumulative_fast = 0
    cumulative_drain = 0
    min_drain_time = np.inf
    
    report = True
    
    completed = False
    for i,(a,t,f,d) in enumerate(zip(arrival_sequence,transfer_sequence,fast_sequence,drain_sequence)):
        if flow:
            t = min(t,arrival_buffer[i]+a)
            f = min(f,intermediate_buffer[i]+t)
            d = min(d,drain_buffer[i]+f)
        else:
            t = min(t,arrival_buffer[i])
            f = min(f,intermediate_buffer[i])
            d = min(d,drain_buffer[i])
            
        drain_time_1,drain_time_2 = compute_draining_times(arrival_buffer[i],
                                                           intermediate_buffer[i],
                                                           drain_buffer[i])
        
        min_drain_time = min(min_drain_time,max(drain_time_1+i,drain_time_2+i))
        min_drain_time = max(drain_time_1,drain_time_2)
            
            
        if (drain_buffer[i] == 0 or intermediate_buffer[i] < ss_thres) and arrival_buffer[i] > 0:
            d = 0
        else:
            t = 0
            
        #if intermediate_buffer[i] > 10:# and drain_time_1 < min_drain_time:
         #   t = 0
            
        assert d+t <= 1
        
        if drain_buffer[i]-intermediate_buffer[i] > h_thres:# and drain_time_2 < min_drain_time:
            f = 0
            
        #if drain_buffer[i] > max(1,h_thres):
         #   print(drain_buffer[i])
          #  f = 0

        
        arrival_buffer[i+1] = max(0,arrival_buffer[i]+a-t)
        intermediate_buffer[i+1] = max(0,intermediate_buffer[i]+t-f)
        drain_buffer[i+1] = max(0,drain_buffer[i]+f-d)
        
        cumulative_arrivals += a
        cumulative_transfer += t
        cumulative_fast += f
        cumulative_drain += d
    if report:
        #if False:#i == int(len(arrival_sequence)/4):
        print(cumulative_transfer/cumulative_arrivals)
        print(cumulative_fast/cumulative_arrivals)
        print(cumulative_drain/cumulative_arrivals)
        print()
        
    #print(n_idles,idling_benefit,idling_benefit_2)
    return arrival_buffer,intermediate_buffer,drain_buffer

In [ ]:
c1 = 1
c2 = 2
c3 = 1

In [ ]:
c1 = 1.5
c2 = 1
c3 = 2

In [ ]:
c1 = 0.1
c2 = 1
c3 = 1

In [ ]:
costs = {}
betas = {}
sc_ratios = {}
eff_rates = {}
slopes = {}
hedging_levels = {}

In [ ]:
percentile = 4

In [ ]:
hedging = np.concatenate((np.arange(0,20,2),np.arange(20,150,10)))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
hedging = np.arange(2,40,2)

In [ ]:
arrivals = []

In [ ]:
#scale_list = [0.1,0.3,1,3]
#scale_list = [0.2,0.4,0.5,0.6,0.7]
scale_list = np.arange(0.35,0.37,0.001)
scale_list = np.arange(0.32,0.333,0.001)
scale_list = np.arange(0.335,0.345,0.001)
scale_list = [0.33]

hedging = np.concatenate((np.arange(0,20,2),np.arange(20,150,10)))
#hedging = np.arange(0,150,10)
hedging = np.arange(50,600,50)
hedging = np.arange(5,100,5)
#hedging = np.arange(7,8,1)
#hedging = [beta_h]
#hedging = np.arange(30,200,10)
#hedging = np.arange(20,500,50)
#hedging = np.concatenate((np.arange(50,500,50),np.arange(500,11000,2000)))
hedging = np.arange(100,11000,1000)
#hedging = np.arange(2,100,5)
#hedging = np.arange(0,100,5)
#offset = -100000
#hedging = [100]

# settings for scale = 3
dur_star = 10000
omega_star = 7.5645
#init_state_star = 210000

#dur_star = int(4500000*1)
duration = dur_star

for scale in reversed(scale_list):
    print(scale)
    scale_costs = []
    scale_rates = []
    
    #init_state = 7e4*scale
    
    mu_demand = 0.33
    mu_drain = mu_transfer = 0.35*2
    mu_fast = 0.34
    
    slack_capacity_h = mu_fast-mu_drain/2
    std_h = np.sqrt(mu_drain*(1-mu_drain)+mu_fast*(1-mu_fast))
    
    omega_h = std_h/slack_capacity_h
    print(slack_capacity_h,std_h,omega_h)
    print()
    
    
    slack_capacity_ss = mu_fast-mu_drain
    std_ss = np.sqrt(mu_fast*(1-mu_fast)+mu_drain*(1-mu_drain))
    
    omega_ss = std_ss/slack_capacity_ss
    
    
    duration = int(1000000 * 1.5 * 0.5)
    print(scale,duration)
    #print(scale,omega)
    #continue
    
    #print(omega/omega_star)
    
    #duration = int((omega/omega_star)**2*dur_star)
    init_state = 10000
    #init_state = 0
    n_seeds = 1#100
    
    beta_h = (1/4)*(percentile**2)*omega_h# + slack_capacity/std
    beta_ss = (1/4)*(percentile**2)*omega_ss
    scaling_ratio = compute_scaling_ratio(mu_drain,mu_demand,std_h,init_state)
    print(scaling_ratio)
    
    hedge = True
    
    for h in reversed(hedging):
        print(h)
        if hedge:
            h_thres = h
            ss_thres = mu_drain+beta_ss*std_ss
        else:
            h_thres = beta_h*std_ss
            ss_thres = mu_drain+h*std_ss
        print(h_thres)
        #thres = 2*mu_drain+h*np.sqrt(mu_drain+mu_fast)
        #thres = h*10
        buf_1_samples = []
        buf_2_samples = []
        buf_3_samples = []
        

        np.random.seed(7)
        
        for _ in range(n_seeds):    

            demand_seq = np.random.binomial(1,mu_demand,duration)
            transfer_seq = np.random.binomial(1,mu_transfer,duration)
            fast_seq = np.random.binomial(1,mu_fast,duration)
            drain_seq = np.random.binomial(1,mu_drain,duration)
        
            arrival_buffer,inter_buffer,drain_buffer = simulate_simple_reentrant_line(
                demand_seq[:duration],
                transfer_seq[:duration],
                fast_seq[:duration],
                drain_seq[:duration],
                h_thres=h_thres,
                ss_thres=5,
                init_state=init_state,
                flow=False,
                init_with_zeros=False)
                
            #try:
             #   end = np.where((arrival_buffer < 10) & (inter_buffer < 10))[0][0]
            #except:
            end = len(arrival_buffer)
            
            buf_1_samples.append(sum(arrival_buffer[0:end]*c1))
            buf_2_samples.append(sum(inter_buffer[0:end]*c2))
            buf_3_samples.append(sum(drain_buffer[0:end]*c3))
            
            #arrivals.append(arrival_buffer)
            
            
        scale_costs.append((np.mean(buf_1_samples),np.mean(buf_2_samples),np.mean(buf_3_samples)))
        #scale_rates.append(zeta*mu_transfer)
        #scale_costs.append(sum(arrival_buffer*c1))
        '''
        a,b = np.histogram(inter_buffer,bins=40,normed=True)
        b = b.reshape(-1,1)
        clf = LinearRegression()
        clf.fit(b[:-15,:],np.log(a[:-14]))
        plt.plot(b[:-15],np.log(a[:-14]),label=scale)
        slopes[scale] = clf.coef_
        '''
        
    costs[scale] = np.array(scale_costs[::-1])
    betas[scale] = beta_h
    sc_ratios[scale] = scaling_ratio
    eff_rates[scale] = np.array(scale_rates[::-1])
plt.legend()

In [ ]:
costs

In [ ]:
#arrivals_2 = arrivals
plt.plot(np.cumsum(np.array(arrivals_10).mean(axis=0)))
plt.plot(np.cumsum(np.array(arrivals).mean(axis=0)),"r")

In [ ]:
#arrivals_10 = arrivals
#plt.plot(np.array(arrivals_30).mean(axis=0)[:2000])
plt.plot(np.array(arrivals_10).mean(axis=0)[:200000])
plt.plot(np.array(arrivals).mean(axis=0)[:20000],"r")

In [ ]:
no_h_cost = ref_cost

In [ ]:
no_h_cost

In [ ]:
min_t_cost/no_h_cost

In [ ]:
no_h_cost/min_t_cost

In [ ]:
bad_cost = ref_cost

In [ ]:
bad_cost/ref_cost

In [ ]:
scale = 0.33

beta = beta_ss#betas[scale]
sc_ratio = sc_ratios[scale]
cost_1,cost_2,cost_3 = zip(*costs[scale])
cost_1=np.array(cost_1)
cost_2=np.array(cost_2)
cost_3=np.array(cost_3)
t_cost = np.array(cost_1)+np.array(cost_2)+np.array(cost_3)
#t_cost = np.array(cost_2)+np.array(cost_3)
#t_cost = np.array(cost_3)
min_t_cost = min(t_cost)
#t_cost = t_cost/min_t_cost
#ref_cost = no_ss_cost
ref_cost = min_t_cost
#ref_cost = no_h_cost

t_cost = t_cost/ref_cost

cost_1=np.array(cost_1)/ref_cost
cost_2=np.array(cost_2)/ref_cost
cost_3=np.array(cost_3)/ref_cost

indexes = np.where(t_cost < 100)[0]

plt.figure(figsize=(16,8))

plt.plot(hedging[indexes],cost_1[indexes],label="Buffer 1 cost")
#plt.plot(hedging[indexes],cost_1[indexes],"o")
#plt.plot(hedging[indexes],cost_2[indexes])
plt.fill_between(hedging[indexes],cost_1[indexes]+cost_2[indexes],cost_1[indexes],alpha=0.5,label="Buffer 2 cost")
plt.fill_between(hedging[indexes],t_cost[indexes],cost_1[indexes]+cost_2[indexes],alpha=0.5, label="Buffer 3 cost")

plt.plot(hedging[indexes],t_cost[indexes],label="Total cost")
plt.plot(hedging[indexes],t_cost[indexes],".")

#plt.vlines(10,min(t_cost[indexes]),max(t_cost[indexes]),label="empirical hedging")
plt.hlines(1.03,min(hedging[indexes]),max(hedging[indexes]),color="r",label="+3% margin")
#plt.hlines(0.97,min(hedging[indexes]),max(hedging[indexes]),color="r",label="-3% margin")
#plt.title("{:.3f}".format(sc_ratio))
plt.ylabel("Relative cumulative cost")
plt.xlabel("Hedging threshold h2")
plt.legend()

In [ ]:
set(np.array([1,2]))

In [ ]:
scale = 0.33

beta = beta_ss#betas[scale]
sc_ratio = sc_ratios[scale]
cost_1,cost_2,cost_3 = zip(*costs[scale])
cost_1=np.array(cost_1)
cost_2=np.array(cost_2)
cost_3=np.array(cost_3)
t_cost = np.array(cost_1)+np.array(cost_2)+np.array(cost_3)
#t_cost = np.array(cost_2)+np.array(cost_3)
#t_cost = np.array(cost_3)
min_t_cost = min(t_cost)
#t_cost = t_cost/min_t_cost
#ref_cost = no_ss_cost
#ref_cost = min_t_cost

t_cost = t_cost/ref_cost

cost_1=np.array(cost_1)/ref_cost
cost_2=np.array(cost_2)/ref_cost
cost_3=np.array(cost_3)/ref_cost

indexes = np.where(t_cost < 100)[0]

plt.figure(figsize=(12,8))

plt.plot(hedging[indexes],cost_1[indexes],label="Buffer 1 cost")
#plt.plot(hedging[indexes],cost_1[indexes],"o")
#plt.plot(hedging[indexes],cost_2[indexes])
plt.fill_between(hedging[indexes],cost_1[indexes]+cost_2[indexes],cost_1[indexes],alpha=0.5,label="Buffer 2 cost")
plt.fill_between(hedging[indexes],t_cost[indexes],cost_1[indexes]+cost_2[indexes],alpha=0.5, label="Buffer 3 cost")

plt.plot(hedging[indexes],t_cost[indexes],label="Total cost")
plt.plot(hedging[indexes],t_cost[indexes],".")

#plt.vlines(10,min(t_cost[indexes]),max(t_cost[indexes]),label="empirical hedging")
plt.hlines(1.03*min(t_cost),min(hedging[indexes]),max(hedging[indexes]),color="r",label="+3% margin")
#plt.hlines(0.97,min(hedging[indexes]),max(hedging[indexes]),color="r",label="-3% margin")
#plt.title("{:.3f}".format(sc_ratio))
plt.ylabel("Relative cumulative cost")
plt.xlabel("Hedging threshold")
plt.legend()

In [ ]:
ref_cost

In [ ]:
scale = 0.33

beta = beta_ss#betas[scale]
sc_ratio = sc_ratios[scale]
cost_1,cost_2,cost_3 = zip(*costs[scale])
cost_1=np.array(cost_1)
cost_2=np.array(cost_2)
cost_3=np.array(cost_3)
t_cost = np.array(cost_1)+np.array(cost_2)+np.array(cost_3)
t_cost = np.array(cost_2)+np.array(cost_3)
#t_cost = np.array(cost_3)
min_t_cost = min(t_cost)
#t_cost = t_cost/min_t_cost
#ref_cost = no_ss_cost
ref_cost = min_t_cost

t_cost = t_cost/ref_cost

cost_1=np.array(cost_1)/ref_cost
cost_2=np.array(cost_2)/ref_cost
cost_3=np.array(cost_3)/ref_cost

indexes = np.where(t_cost < 100)[0]

plt.figure(figsize=(12,4))

#plt.plot(hedging[indexes],cost_1[indexes],label="Buffer 1 cost")
#plt.plot(hedging[indexes],cost_1[indexes],"o")
#plt.plot(hedging[indexes],cost_2[indexes])
#plt.fill_between(hedging[indexes],cost_1[indexes]+cost_2[indexes],cost_1[indexes],alpha=0.5,label="Buffer 2 cost")
#plt.fill_between(hedging[indexes],t_cost[indexes],cost_1[indexes]+cost_2[indexes],alpha=0.5, label="Buffer 3 cost")

plt.plot(hedging[indexes],t_cost[indexes],label="Total cost")
plt.plot(hedging[indexes],t_cost[indexes],".")

#plt.vlines(10,min(t_cost[indexes]),max(t_cost[indexes]),label="empirical hedging")
plt.hlines(1.03,min(hedging[indexes]),max(hedging[indexes]),color="r",label="+3% margin")
#plt.hlines(0.97,min(hedging[indexes]),max(hedging[indexes]),color="r",label="-3% margin")
#plt.title("{:.3f}".format(sc_ratio))
plt.ylabel("Relative cumulative cost")
plt.xlabel("Hedging threshold")
plt.legend()

In [ ]:
(2120/(1-0.33/0.345))/(2770/(1-0.33/0.35))

In [ ]:
np.sum(costs[0.33])/no_ss_cost

In [ ]:
no_ss_cost = np.sum(costs[0.33])

In [ ]:
no_ss_cost

In [ ]:
plt.plot(inter_buffer[:10000], label="buffer 3")

In [ ]:
np.sum(inter_buffer == 0)

In [ ]:
np.sum(inter_buffer == 0)

In [ ]:
-1.02*2977.9+1.05*2874.3

In [ ]:
-1.02*2972.+1.05*2868.6

In [ ]:
2874.3*0.35,2868.6*0.35

In [ ]:
988+18,983+21

In [ ]:
2/0.35

In [ ]:
plt.plot(inter_buffer[8000:10000], label="buffer 3")

In [ ]:
end = 100000
plt.figure(figsize=(16,6))
#plt.plot(arrival_buffer[:end],label="buffer 1")
plt.plot(inter_buffer[30000:end], label="buffer 2")
plt.plot(drain_buffer[30000:end], label="buffer 3")
plt.legend()

In [ ]:
plt.hist(inter_buffer,bins=np.arange(150))
plt.hist(drain_buffer,bins=np.arange(150))

In [ ]:
end = 80000
plt.figure(figsize=(16,6))
#plt.plot(arrival_buffer[:end],label="buffer 1")
plt.plot(inter_buffer[:end], label="buffer 2")
#plt.plot(drain_buffer[:end], label="buffer 3")
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(arrival_buffer,label="buffer 1")
plt.plot(inter_buffer, label="buffer 2")
plt.plot(drain_buffer, label="buffer 3")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(arrival_buffer,label="buffer 1")
plt.plot(inter_buffer, label="buffer 2")
plt.plot(drain_buffer, label="buffer 3")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
plt.legend()

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(arrival_buffer,label="buffer 1")
plt.plot(inter_buffer, label="buffer 2")
plt.plot(drain_buffer, label="buffer 3")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
plt.legend()

In [ ]:
f,ax = plt.subplots(2,1,figsize=(16,10))
ax[0].plot(arrival_buffer,label="buffer 1")
ax[0].plot(inter_buffer, label="buffer 2")
ax[0].plot(drain_buffer, label="buffer 3")
ax[0].set_ylabel("Buffer level")
ax[0].legend()

drain_time_1,drain_time_2=compute_draining_times(arrival_buffer,inter_buffer,drain_buffer)

ax[1].plot(drain_time_1,label="resource 1")
ax[1].plot(drain_time_2,label="resource 2")
ax[1].set_ylabel("Draining time")
ax[1].legend()
#ax[1].gca().set_aspect("equal")


In [ ]:
drain_time_1,drain_time_2=compute_draining_times(arrival_buffer,inter_buffer,drain_buffer)

In [ ]:
workload_1,workload_2 = compute_workloads(arrival_buffer,inter_buffer,drain_buffer)

In [ ]:
np.array([i for i in range(10)])

In [ ]:
np.where(np.array([i for i in range(10)]) > 5)[0]

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(drain_time_1,label="1")
plt.plot(drain_time_2)
plt.legend()
plt.gca().set_aspect("equal")

In [ ]:
plt.plot(workload_1)
plt.plot(workload_2)

In [ ]:
#plt.figure(figsize=(16,6))
f,ax = plt.subplots(2,1,figsize=(16,8))
ax[0].plot(arrival_buffer,label="buffer 1")
ax[0].plot(inter_buffer, label="buffer 2")
ax[0].plot(drain_buffer, label="buffer 3")
ax[1].plot(arrival_buffer*c1+inter_buffer*c2+drain_buffer*c3,label="Total cost")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
ax[0].legend()
ax[1].legend()

In [ ]:
#plt.figure(figsize=(16,6))
f,ax = plt.subplots(2,1,figsize=(16,8))
ax[0].plot(arrival_buffer,label="buffer 1")
ax[0].plot(inter_buffer, label="buffer 2")
ax[0].plot(drain_buffer, label="buffer 3")
ax[1].plot(arrival_buffer*c1+inter_buffer*c2+drain_buffer*c3,label="Total cost")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
ax[0].legend()
ax[1].legend()

In [ ]:
cost_2 = arrival_buffer*c1+inter_buffer*c2+drain_buffer*c3

In [ ]:
plt.plot(arrival_buffer*c1+inter_buffer*c2+drain_buffer*c3)
plt.plot(cost_2)

In [ ]:
plt.plot(cost_1)
plt.plot(cost_2)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(arrival_buffer,label="buffer 1")
plt.plot(inter_buffer, label="buffer 2")
plt.plot(drain_buffer, label="buffer 3")
#plt.hlines(3,0,15000, label = "ss")
#plt.hlines(5,0,15000, label = "ss")
plt.legend()

In [ ]:
workload = arrival_buffer/(mu_drain/2)+(inter_buffer+drain_buffer)/(mu_drain)

In [ ]:
workload_2 = (inter_buffer+arrival_buffer)/(mu_fast)

In [ ]:
plt.plot(workload[:100000],workload_2[:100000])

In [ ]:
plt.plot(workload[:100000],workload_2[:100000])

In [ ]:
min_drain_time = workload/(1-mu_demand*2/mu_drain)

In [ ]:
np.mean(min_drain_time),np.median(min_drain_time)

In [ ]:
np.mean(min_drain_time > 1000)

In [ ]:
a,b,_ = plt.hist(min_drain_time,bins=np.arange(0,14000,50),normed=True)

In [ ]:
np.argmax(a)

In [ ]:
a[:20]

In [ ]:
b[:20]

In [ ]:
b[17]

In [ ]:
np.mean(arrival_buffer)

In [ ]:
np.mean(inter_buffer)

In [ ]:
plt.figure(figsize=(10,8))
dur = np.arange(54000,65000)
#dur = np.arange(300000)
plt.fill_between(dur,drain_buffer[dur],label = "buffer 3")
#plt.plot(dur,drain_buffer[dur])
plt.fill_between(dur,-inter_buffer[dur],label='-buffer 2')
#plt.fill_between(dur,-inter_buffer[dur],np.minimum(-inter_buffer[dur],-offset),label='-buffer 2')
#plt.plot(dur,-inter_buffer[dur])
#plt.plot(dur,a[dur]-offset,"k",alpha=0.5)
plt.ylim(top=50,bottom=-100)
plt.legend()

In [ ]:
np.mean(arrival_buffer)

In [ ]:
a = drain_buffer

In [ ]:
std_h

In [ ]:
np.percentile(inter_buffer,33)

In [ ]:
350*0.16

In [ ]:
inter_buffer_ss = inter_buffer

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(inter_buffer,bins=np.arange(150),normed=True,label="long drain")

plt.vlines(np.percentile(inter_buffer,33),0,0.04,label="long_drain")

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(inter_buffer,bins=np.arange(150),normed=True,label="long drain")
plt.hist(inter_buffer_ss,bins=np.arange(150),normed=True,label="steady state",alpha=0.7)

plt.xlabel("Buffer 2 level")
plt.ylabel("Occupancy probability")

h = np.percentile(inter_buffer,33)

plt.vlines(np.percentile(inter_buffer,33),0,0.04,label="long_drain")
plt.vlines(np.percentile(inter_buffer_ss,33),0,0.04,label="steady state",color="r")
plt.legend()

In [ ]:
np.percentile(150-drain_buffer,33)

In [ ]:
1/(omega_h*std_h)

In [ ]:
plt.plot(drain_buffer)

In [ ]:
-np.log(0.33)/(0.01*3.5)

In [ ]:
b,a = zip(*slopes.items())

In [ ]:
clf = LinearRegression()
clf.fit(np.array(b).reshape(-1,1),a)

In [ ]:
clf.coef_

In [ ]:
plt.plot(np.array(b),a,".")
plt.plot(np.array(b),clf.predict(np.array(b).reshape(-1,1)))

In [ ]:
np.histogram(inter_buffer,bins=50)

In [ ]:
beta_ss = (1/4)*(percentile**2)*omega_ss

In [ ]:
beta_ss

In [ ]:
mu_demand,mu_transfer,mu_fast,mu_drain

In [ ]:
std_h**2*(1-omega_h*2*(c3/c2))/(4*slack_capacity_h)

In [ ]:
plt.plot(arrival_buffer[:1000000])

In [ ]:
np.sum(drain_buffer)/(26*len(drain_buffer))

In [ ]:
#
#plt.plot(arrival_buffer[:1000000])
#plt.plot(inter_buffer[:1000000])
plt.plot(drain_buffer[:1000000])
plt.plot(inter_buffer[:1000000],label='safety stocks')
plt.legend()

In [ ]:
#
#plt.plot(arrival_buffer[:1000000])
#plt.plot(inter_buffer[:1000000])
plt.plot(drain_buffer[:1000000])
plt.plot(inter_buffer[:1000000],label='safety stocks')
plt.legend()

In [ ]:
#
#plt.plot(arrival_buffer[:1000000])
#plt.plot(inter_buffer[:1000000])
#plt.plot(drain_buffer[:1000000])
plt.plot(inter_buffer[:100000000],label='safety stocks')
plt.legend()

In [ ]:
max(drain_buffer)- np.percentile(drain_buffer,66)

In [ ]:
np.percentile(inter_buffer,33)

In [ ]:
plt.plot(inter_buffer)

In [ ]:
plt.plot(np.arange(199,-1,-1),0.035*np.exp(np.arange(200)*-0.035))

In [ ]:
std_h

In [ ]:
(0.7*omega_h*std_h)

In [ ]:
s = 1/(0.7*omega_h*std_h)

In [ ]:
s

In [ ]:
1/clf.coef_

In [ ]:
plt.hist(drain_buffer,bins=40,normed=True)
#plt.plot(b[15:,:],clf.predict(b[15:,:]))


In [ ]:
np.log(0.66)/s

In [ ]:
plt.figure(figsize=(10,6))

a,b,_ = plt.hist(drain_buffer,bins=30,normed=True)

b = b.reshape(-1,1)
clf = LinearRegression()
clf.fit(b[15:,:],np.log(a[14:]))
print(clf.coef_)


#plt.plot(np.arange(149,-1,-1),clf.coef_[0]*np.exp(np.arange(150)*-clf.coef_[0]))
plt.plot(np.arange(149,-1,-1),s*np.exp(np.arange(150)*-s),linewidth=2)

plt.vlines(150+np.log(0.66)/s,0,0.04,color="r")
plt.xlabel("Buffer 3 level")
plt.ylabel("Occupancy probability")

In [ ]:
np.percentile(a,66)

In [ ]:
1/omega_h

In [ ]:
len(a)

In [ ]:
len(b)

In [ ]:
0.33-0.34

In [ ]:
3/200

In [ ]:
mu_demand/mu_fast

In [ ]:
mu_transfer/2/mu_fast

In [ ]:
5/140

In [ ]:
-np.log(1-0.33)/(3.5*0.015)

In [ ]:
plt.plot(b[10:],np.log(a[9:]))

In [ ]:
#
#plt.plot(arrival_buffer[:1000000])
#plt.plot(inter_buffer[:1000000])
plt.plot(-drain_buffer[:1000000])
plt.plot(inter_buffer[:1000000],label='safety stocks')
plt.legend()

In [ ]:
beta_h*std_h/(beta_ss*std_ss)

In [ ]:
beta_h

In [ ]:
plt.figure(figsize=(14,8))
run = np.arange(10000)
plt.fill_between(run,inter_buffer[run],label="buffer 2")
plt.fill_between(run,drain_buffer[run],label="buffer 3")
plt.legend()

In [ ]:
omega_h

In [ ]:
cost_3

In [ ]:
scale = 0.33

beta = beta_ss#betas[scale]
sc_ratio = sc_ratios[scale]
cost_1,cost_2,cost_3 = zip(*costs[scale])
cost_1=np.array(cost_1)
cost_2=np.array(cost_2)
cost_3=np.array(cost_3)
t_cost = np.array(cost_1)+np.array(cost_2)+np.array(cost_3)
min_t_cost = min(t_cost)
t_cost = t_cost/min_t_cost

cost_1=np.array(cost_1)/min_t_cost
cost_2=np.array(cost_2)/min_t_cost
cost_3=np.array(cost_3)/min_t_cost

indexes = np.where(t_cost < 5)[0]

plt.figure(figsize=(12,8))

plt.plot(hedging[indexes],cost_1[indexes],label="Buffer 1 cost")
#plt.plot(hedging[indexes],cost_1[indexes],"o")
#plt.plot(hedging[indexes],cost_2[indexes])
plt.fill_between(hedging[indexes],cost_1[indexes]+cost_2[indexes],cost_1[indexes],alpha=0.1)
plt.fill_between(hedging[indexes],t_cost[indexes],cost_1[indexes]+cost_2[indexes],alpha=0.1)
plt.plot(hedging[indexes],t_cost[indexes],label="Total cost")
plt.plot(hedging[indexes],t_cost[indexes],".")
plt.vlines(beta,min(t_cost[indexes]),max(t_cost[indexes]),label="beta")
plt.hlines(1.03,min(hedging[indexes]),max(hedging[indexes]),color="r",label="3% margin")
plt.title("{:.3f}".format(sc_ratio))
plt.ylabel("Relative cumulative cost")
plt.xlabel("Threshold (xSTD)")
plt.legend()

In [ ]:
scale = 0.33

beta = betas[scale]
sc_ratio = sc_ratios[scale]
cost_1,cost_2,cost_3 = zip(*costs[scale])
cost_1=np.array(cost_1)
cost_2=np.array(cost_2)
cost_3=np.array(cost_3)
t_cost = np.array(cost_1)+np.array(cost_2)+np.array(cost_3)
min_t_cost = min(t_cost)
t_cost = t_cost/min_t_cost

cost_1=np.array(cost_1)/min_t_cost
cost_2=np.array(cost_2)/min_t_cost
cost_3=np.array(cost_3)/min_t_cost

indexes = np.where(t_cost < 2e6)[0]

plt.figure(figsize=(12,8))

plt.plot(hedging[indexes],cost_1[indexes],label="Buffer 1 cost")
#plt.plot(hedging[indexes],cost_1[indexes],"o")
#plt.plot(hedging[indexes],cost_2[indexes])
plt.fill_between(hedging[indexes],cost_1[indexes]+cost_2[indexes],cost_1[indexes],alpha=0.1)
plt.fill_between(hedging[indexes],t_cost[indexes],cost_1[indexes]+cost_2[indexes],alpha=0.1)
plt.plot(hedging[indexes],t_cost[indexes],label="Total cost")
plt.plot(hedging[indexes],t_cost[indexes],".")
plt.vlines(beta,min(t_cost[indexes]),max(t_cost[indexes]),label="beta")
plt.hlines(1.03,min(hedging[indexes]),max(hedging[indexes]),color="r",label="3% margin")
plt.title("{:.3f}".format(sc_ratio))
plt.ylabel("Relative cumulative cost")
plt.xlabel("Threshold (xSTD)")
plt.legend()

In [ ]:
scale = 3
beta = betas[scale]
sc_ratio = sc_ratios[scale]
cost = costs[scale]
r_cost = cost/min(cost)
indexes = np.where(r_cost < 1.2)[0]
plt.plot(hedging[indexes],r_cost[indexes])
plt.plot(hedging[indexes],r_cost[indexes],".")
plt.vlines(beta,min(r_cost[indexes]),max(r_cost[indexes]))
plt.hlines(1.03,min(hedging[indexes]),max(hedging[indexes]),color="r")
plt.title("{:.3f}".format(sc_ratio))

In [ ]:
plt.plot(hedging,costs[1])

In [ ]:
mu_demand

In [ ]:
percentile = 3.1

In [ ]:
scale = 0.1

In [ ]:
cost = []
rates = []
hedging = np.arange(30,200,100)

f,ax = plt.subplots(3,1,figsize=(16,8))

duration = 10000
plot_range = range(0,duration)

mu_demand = 30*scale
mu_drain = mu_demand*1.02
mu_transfer = mu_drain + (mu_drain-mu_demand)*1

slack_capacity = mu_transfer-mu_drain
std = np.sqrt(mu_drain+mu_transfer)

omega = std/slack_capacity

beta = (1/4)*(percentile**2)*(std/slack_capacity)

hedging=[beta/4,beta/2,beta]
#hedging=[beta]

init_state = (mu_drain-mu_demand)*duration*0.6


np.random.seed(5)

demand_seq = np.random.poisson(mu_demand,duration)
transfer_seq = np.random.poisson(mu_transfer,duration)
drain_seq = np.random.poisson(mu_drain,duration)

cumul =False
for h in reversed(hedging):
    thres = 2*mu_drain+h*np.sqrt(mu_drain+mu_transfer)
    
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk_repeat(
        demand_seq[:duration],
        transfer_seq[:duration],
        drain_seq[:duration],
        thres,
        init_state=init_state,
    flow=False)
    
    #print(np.where(drain_buffer == 0))
    
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    rates.append(zeta*mu_transfer)
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer)[plot_range],label=int(h))
        ax[0].plot(np.cumsum(arrival_buffer)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer)[plot_range])
        #ax[1].plot(np.ones(len(plot_range))*thres,".-")
        ax[0].plot((arrival_buffer)[plot_range],label="{} * {}".format(int(h),int(std)))
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
    
ax[0].set_ylabel("Items in buffer 1")
ax[1].set_ylabel("Items in buffer 2")
ax[2].set_ylabel("Total cost")
f.legend()

In [ ]:
slack_capacity

In [ ]:
std/slack_capacity

In [ ]:
mu_drain*c2

In [ ]:
thres*c2

In [ ]:
np.sum(drain_buffer == 0)

In [ ]:
mu_demand

In [ ]:
rates

In [ ]:
mu_demand

In [ ]:
mu_transfer

In [ ]:
time_horizon

In [ ]:
offset/std

In [ ]:
offset

In [ ]:
percentile = 1.645
#percentile = 0
percentile = 1.96
#percentile = 2.33
percentile = 3.1
#percentile = 1
#percentile = 7
slack_capacity = mu_transfer-mu_drain
std = np.sqrt(mu_drain+mu_transfer)
time_horizon = (percentile*std)**2/(2*slack_capacity)**2
offset = time_horizon*(-slack_capacity) + percentile*std*np.sqrt(time_horizon)
time_horizon = int(np.ceil(time_horizon))
offset = int(np.ceil(offset))

In [ ]:
percentile*np.sqrt(3)

In [ ]:
slack_capacity = mu_transfer-mu_drain
std = np.sqrt(mu_drain+mu_transfer)
beta = (1/4)*(percentile**2)*(std/slack_capacity) + slack_capacity/std

In [ ]:
offset

In [ ]:
std

In [ ]:
slack_capacity

In [ ]:
slack_capacity/std

In [ ]:
slack_capacity

In [ ]:
0.5*percentile*std/np.sqrt(time_horizon)

In [ ]:
offset/std + slack_capacity/std

In [ ]:
scaling_ratio = compute_scaling_ratio(mu_drain,mu_demand,std,init_state)

In [ ]:
beta

In [ ]:
min_cost = min(cost)
hedging = np.array(hedging)
r_cost = np.array([c/min_cost for c in cost[::-1]])

indexes = np.where(r_cost < 1.2)[0]
plt.plot(hedging[indexes],r_cost[indexes])
plt.plot(hedging[indexes],r_cost[indexes],".")
plt.vlines(beta,min(r_cost[indexes]),max(r_cost[indexes]))
plt.title("{:.3f}".format(scaling_ratio))

In [ ]:
min_cost = min(cost)
hedging = np.array(hedging)
r_cost = np.array([c/min_cost for c in cost[::-1]])

indexes = np.where(r_cost < 1.2)[0]
plt.plot(hedging[indexes],r_cost[indexes])
plt.plot(hedging[indexes],r_cost[indexes],".")
plt.vlines(beta,min(r_cost[indexes]),max(r_cost[indexes]))
plt.title("{:.3f}".format(scaling_ratio))

In [ ]:
cost = []
hedging = np.arange(30,60,5)
init_state = 7e4
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 100000
plot_range = range(0,10000)
plot_range = range(0,200)
cumul =False
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain+mu_transfer)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk_repeat(demand_seq[:duration],
                                                                             transfer_seq[:duration],
                                                                             drain_seq[:duration],
                                                                             thres,init_state=init_state,
                                                                            flow=False)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.legend()

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
cost = []
hedging = np.arange(5,70,5)
init_state = 1e4
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 6000
plot_range = range(0,6000)
#plot_range = range(0,300)
cumul =False
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
    
thres = 1e6
#thres = h*10
arrival_buffer,drain_buffer,_ = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
#plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
if cumul:
    #ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label="e")
    ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range],label="e")
    #ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
else:
    #ax[1].plot((drain_buffer*c2)[plot_range],label="e")
    ax[0].plot((arrival_buffer*c1)[plot_range],label="e")
    #ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
f.legend()

In [ ]:
(mu_transfer-mu_demand)/((zeta*mu_transfer)-mu_demand)

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
h = []
for i in np.arange(0.94,0.949,0.001):
    h.append(1/(1-i))
    
plt.plot(np.arange(0.94,0.949,0.001)/0.94,[i/min(h) for i in h])

In [ ]:
min_cost = min(cost)

In [ ]:
cost[0]-cost[1]

In [ ]:
plt.plot(drain_buffer[:300])

In [ ]:
plt.plot(arrival_buffer[:600])

In [ ]:
plt.plot(buffer_seq[:1000])

In [ ]:
sum(buffer_seq)

In [ ]:
sum(buffer_seq)

In [ ]:
np.percentile((supply_seq-demand_seq)[(supply_seq-demand_seq) < 0],0.01)

In [ ]:
plt.plot(np.cumsum(supply_seq)-np.cumsum(demand_seq))

In [ ]:
percentile = 1.645
#percentile = 0
#percentile = 1.96
#percentile = 2.33
slack_capacity = mu_supply-mu_demand
time_horizon = (percentile**2)*mu_supply/(2*slack_capacity**2)
offset = time_horizon*(-slack_capacity) + percentile* np.sqrt(mu_supply*2*time_horizon)
print(time_horizon*2)
time_horizon = int(np.ceil(time_horizon))
offset = int(np.ceil(offset))

In [ ]:
time_horizon = (percentile**2)*mu_supply*2/slack_capacity**2
time_horizon = int(np.ceil(time_horizon))

In [ ]:
y = []
for d in range(time_horizon):
    y.append(d*(slack_capacity) - percentile* np.sqrt(mu_supply*2*d))

In [ ]:
y_1 = y
time_horizon_1 = time_horizon

In [ ]:
y_2 = y
time_horizon_2 = time_horizon

In [ ]:
time_horizon/time_horizon_1

In [ ]:
1.96/1.645

In [ ]:
plt.plot(range(time_horizon),y)
plt.plot(range(time_horizon_1),y_1)
plt.plot(range(time_horizon_2),y_2)

In [ ]:
y

In [ ]:
time_horizon

In [ ]:
offset

In [ ]:
thres = poisson.ppf(0.95,mu_demand)
#thres = 0
thres = poisson.ppf(0.5,mu_demand)

def idle_supply(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if inv_pos > thres+offset:
            s = 0
            idle_count += 1
        idle_supply_seq[i] = s
        inv_pos += s-d
    #print(idle_count/len(supply_seq)) 
    return idle_supply_seq

def idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if (inv_pos > thres+offset) and days_count >= time_horizon:
            s = 0
            days_count = 0
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        days_count += 1
    return inv_pos_seq

def idle_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = False
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        if not just_idled:
            days_count += 1
            
    return inv_pos_seq

def work_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = True
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            days_count = 0
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            days_count += 1
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
            
    return inv_pos_seq

def idle_supply_smooth(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    inv_pos_array = np.zeros_like(supply_seq)
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0:
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            idle_count += 1
            
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos = min(inv_pos,offset)
        inv_pos_array[i] = inv_pos

    #print(idle_count/len(supply_seq)) 
    print(inv_pos)
    return inv_pos_array

In [ ]:
slack_capacity/np.sqrt(2*mu_demand)

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+100])

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
slack_capacity

In [ ]:
inv_pos_seq = work_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,41,69)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon)

print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
idle_supply_seq,inv_pos_seq = idle_supply_smooth(demand_seq,supply_seq, np.ceil(offset))
#inv_pos_seq = offset + np.cumsum(idle_supply_seq)-np.cumsum(demand_seq)
print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

plt.plot(inv_pos_seq[:1200])

In [ ]:
n_sims = 100000
demand_sum = np.random.poisson(mu_demand*np.ceil(time_horizon),n_sims)
supply_sum = np.random.poisson(mu_supply*np.ceil(time_horizon),n_sims)

print(np.mean((demand_sum-supply_sum) > np.ceil(offset)))

In [ ]:
offset+time_horizon*slack_capacity

In [ ]:
1001 % 100

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
np.random.seed(500)
n_sims = 100000
#n_sims = 20
stockouts = []
last_day_stockouts = []
last_day_stockouts_vals = []
ave_inventories = []
sim_time_horizon = time_horizon
for i in range(n_sims):
    demand = np.random.poisson(mu_demand,sim_time_horizon)
    supply = np.random.poisson(mu_supply,sim_time_horizon)
    inv_pos_seq = offset + np.cumsum(supply)-np.cumsum(demand)
    stockouts.append(np.sum(inv_pos_seq < 0))
    last_day_stockouts.append(inv_pos_seq[-1] < offset)
    if last_day_stockouts[-1]:
        last_day_stockouts_vals.append(inv_pos_seq[-1]-offset)
    
    ave_inventories.append(np.mean(inv_pos_seq))
    if i % 10000 == 0:
        plt.plot(inv_pos_seq)
    
sum(stockouts)/(sim_time_horizon*n_sims),np.sum(last_day_stockouts)/(n_sims),np.mean(ave_inventories)


In [ ]:
offset

In [ ]:
np.median(last_day_stockouts_vals)

In [ ]:
for offset in range(200):
    stock_out_probs = []
    for d in range(1,time_horizon+1):
        stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
        stock_out_probs.append(stock_out_prob)
    overal_stockout_prob = np.mean(stock_out_probs)
    #print(overal_stockout_prob)
    if overal_stockout_prob < 0.05:
        break

In [ ]:
time_horizon

In [ ]:
def get_percentile_deficit(cycle_dur,slack_capacity,variance,percentile = 0.5):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    cum_percentile = 0
    prev_cum_prob = cum_deficit_prob
    for i in range(10000):
        cum_prob = norm.cdf(-i,mu,std)
        prob = (prev_cum_prob - cum_prob)/cum_deficit_prob
        cum_percentile += prob
        if cum_percentile >= percentile:
            return i
        prev_cum_prob = cum_prob
        
a = get_percentile_deficit(time_horizon/4,slack_capacity,2*mu_supply)
#get_percentile_deficit(slack_capacity,2*mu_supply,time_horizon)
print(a)

def compute_recovery_time(slack_capacity,variance,deficit,bound = 2.33):
    dur = ((bound*np.sqrt(variance)+np.sqrt(bound**2*variance+4*slack_capacity*deficit))/(2*slack_capacity))**2
    return int(np.ceil(dur))

print(compute_recovery_time(slack_capacity,2*mu_supply,a))

def get_average_stockout_prob(duration,slack_capacity,variance,start):
    stock_out_probs = []
    for d in range(1,duration+1):
        stock_out_prob = norm.cdf(0,start+slack_capacity*d,np.sqrt(variance*d))
        stock_out_probs.append(stock_out_prob)
    average_stockout_prob = np.mean(stock_out_probs)
    return average_stockout_prob

def compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    #print(cum_deficit_prob)
    deficit = get_percentile_deficit(cycle_dur,slack_capacity,variance,0.95)
    #print(deficit)
    rec_dur = compute_recovery_time(slack_capacity,variance,deficit)
    #print(rec_dur)
    cycle_stockout_prob = get_average_stockout_prob(cycle_dur,slack_capacity,variance,offset)
    
    rec_dur = int(np.ceil(deficit/slack_capacity))
    print(rec_dur)
    rec_stockout_prob = get_average_stockout_prob(rec_dur,slack_capacity,variance,offset-deficit)
    #print(cycle_stockout_prob,rec_stockout_prob)
    
    effective_duration = (cycle_dur+cum_deficit_prob*rec_dur)
    #print(cycle_dur/effective_duration)
    
    overall_stockout_prob = (cycle_dur*cycle_stockout_prob+cum_deficit_prob*rec_dur*rec_stockout_prob)/effective_duration
    
    overall_inventory_cost = (cycle_dur*(0.5*slack_capacity*cycle_dur+offset)+cum_deficit_prob*rec_dur*(0.5*slack_capacity*rec_dur+offset-deficit))/effective_duration
    #print(overall_inventory_cost)
    return overall_stockout_prob,overall_inventory_cost

In [ ]:
time_horizon/4

In [ ]:
variance = 2*mu_supply

In [ ]:
min_inv_cost = np.inf
min_cycle_dur = None
min_offset = None
for cycle_dur in range(1,int(time_horizon)):

    for offset in range(200):
        overall_stockout_prob,inv_cost = compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset)
        #print(overall_stockout_prob)
        if overall_stockout_prob < 0.05:
            break
    
    print(cycle_dur,inv_cost)
    if inv_cost < min_inv_cost:
        print(cycle_dur)
        min_inv_cost = inv_cost
        min_cycle_dur = cycle_dur
        min_offset = offset
    
print(offset)

In [ ]:
min_offset

In [ ]:
min_cycle_dur

In [ ]:
min_inv_cost

In [ ]:
time_horizon

In [ ]:
int(time_horizon)*(0.5*slack_capacity)

In [ ]:
inv_cost

In [ ]:
print(overal_stockout_prob)

In [ ]:
overal_stockout_prob

In [ ]:
probs = []
deficit = 10000
for i in range(deficit):
    v = -offset-i
    mu = slack_capacity*time_horizon
    std = np.sqrt(2*mu_supply*time_horizon)
    probs.append(norm.cdf(v,mu,std))
    #print(i,probs[-1])
    
np.sum(-np.diff(probs)*np.arange(1,deficit)/norm.cdf(-offset,mu,std))

In [ ]:
offsets = []
for dur in range(1,time_horizon+1):
    for offset in range(200):
        stock_out_probs = []
        for d in range(1,dur+1):
            stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
            stock_out_probs.append(stock_out_prob)
        overal_stockout_prob = np.mean(stock_out_probs)
        #print(overal_stockout_prob)
        if overal_stockout_prob < 0.05:
            break
    #print(dur,offset)
    offsets.append(offset)

In [ ]:
plt.plot(offsets)

In [ ]:
norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
mu

In [ ]:
(-np.diff(probs)/norm.cdf(-offset,mu,std))[:50]

In [ ]:
-np.diff(probs)/norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
np.sum(last_day_stockouts)/(n_sims)

In [ ]:
sum(stockouts)/(int(np.ceil(time_horizon))*n_sims)

In [ ]:
np.sum(last_day_stockouts)

In [ ]:
np.sum(last_day_stockouts)/sum(stockouts)

In [ ]:
np.mean(stockouts)

In [ ]:
stockouts = np.array(stockouts)

In [ ]:
np.median(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts,bins=range(0,50,2))

In [ ]:
2*time_horizon

In [ ]:
norm.cdf(-offset,slack_capacity*10,np.sqrt(mu_supply*10))

In [ ]:
int(np.ceil(time_horizon))